# Replicating SISSO++ Descriptor Discovery with **pySISSO**

This notebook reproduces the descriptor discovery benchmark that compares Rocksalt (RS) and Zincblende (ZB) structural stability in 82 octet binary compounds, originally performed with **SISSO++**. Here we replicate the benchmark using the Python implementation **pySISSO**.

**Requirements**  
- `pysisso` (>=0.4) – `pip install pysisso`  
- `pandas`, `numpy`, `scikit-learn`, `nbformat` (common scientific stack)

Place the dataset `benchmark_data.csv` in the same folder as this notebook or adjust the path below.


## 1. Load and inspect the dataset

In [3]:
import sys
import os

# Add the 'src' directory to the Python path
# This tells Python where to look for the 'pysisso' module
sys.path.insert(0, os.path.abspath('./src'))

# Now your original imports will work
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

from pysisso import SISSORegressor

# Set display options
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
%matplotlib inline

print("✅ 'src' directory added to Python path. You can now import pysisso.")

✅ 'src' directory added to Python path. You can now import pysisso.


### Define target and feature columns

In [5]:
data = pd.read_csv('Final_dataset.csv')

# --- Feature and Target Selection ---
TARGET_PROPERTIES = ['Totel_Energy', 'E_tet', 'E_oct', 'E_a', 'OCV']

# Define all potential primary features based on column names
PRIMARY_FEATURES = [
    'Lattice_Constant_A', 'Lattice_Constant_B', 'Lattice_Constant_X',
    'Ionic_Radii_A', 'Ionic_Radii_B', 'Ionic_Radii_X',
    'EA_A', 'EA_B', 'EA_X',
    'IP_A', 'IP_B', 'IP_X',
    'r_s_A', 'r_s_B', 'r_s_X',
    'r_p_A', 'r_p_B', 'r_p_X',
    'r_d_A', 'r_d_B', 'r_d_X',
    'HOMO_A', 'HOMO_B', 'HOMO_X',
    'LUMO_A', 'LUMO_B', 'LUMO_X'
]

# Ensure all selected features exist in the dataframe after cleaning
PRIMARY_FEATURES = [f for f in PRIMARY_FEATURES if f in data.columns]

X = data[PRIMARY_FEATURES]
y = data[TARGET_PROPERTIES]

print(f"\nDataset prepared for Multi-Task Regression.")
print(f"Number of samples: {len(data)}")
print(f"Number of primary features: {len(X.columns)}")
print(f"Number of target properties: {len(y.columns)}")


Dataset prepared for Multi-Task Regression.
Number of samples: 765
Number of primary features: 27
Number of target properties: 5


## 2. Configure **pySISSO**
The settings below mirror those used in the SISSO++ benchmark: depth = 2, operators = `add`, `sub`, `mul`, descriptor dimensionality up to 3, and Bayesian Information Criterion (BIC) for model selection.

In [6]:
from pysisso import SISSORegressor

config = {
    'task_type': 'regression',
    'depth': 2,
    'op_rules': [ {'op': 'add'}, {'op': 'sub'}, {'op': 'mul'} ],
    'interaction_only': False,
    'max_D': 3,
    'sis_sizes': [100, 50],
    'search_strategy': 'sisso++',
    'max_feat_cross_correlation': 0.95,
    'loss': 'l2',
    'alpha': 1e-5,
    'selection_method': 'bic',
    'cv': -1,
    'fix_intercept': False,
    'n_jobs': -1,
    'random_state': 42
}

sisso = SISSORegressor(**config)

## 3. Run descriptor search

In [7]:
%%time
sisso.fit(X, y)


--- STARTING ITERATIVE FEATURE GENERATION & SCREENING ---

Starting iterative feature generation on CPU to depth 2...
  Keeping top 100 features per iteration.

  Depth 1: Generating new features...
    Generated 630 new unique features. Total candidates: 657
  Screening and pruning 657 candidates...
Running SIS to screen 657 features...
  Using average correlation for multi-task screening.
  CPU SIS screening complete. Top feature: '(Ionic_Radii_X-r_s_B)'
    Pruned feature space to 100 candidates.

  Depth 2: Generating new features...
    Generated 9449 new unique features. Total candidates: 9549
  Screening and pruning 9549 candidates...
Running SIS to screen 9549 features...
  Using average correlation for multi-task screening.
  CPU SIS screening complete. Top feature: '((r_s_B-Ionic_Radii_B)-(r_s_X*r_s_X))'
    Pruned feature space to 100 candidates.

Iterative feature generation complete.

Final feature space size after iterative screening: 100

==================== Starting S

RuntimeError: L0 search found no models from the iteratively generated feature space.

### Best descriptor dimension

In [ ]:
print('Selected best descriptor dimension via BIC: D =', sisso.best_D_)

### Descriptor formulas for each dimension

In [ ]:
from pysisso import utils as sis_utils

for D, model_info in sisso.models_by_dim_.items():
    formula_str = sis_utils.print_descriptor_formula(
        model_info['sym_features'], model_info.get('coef'),
        sisso.task_type_, sisso.fix_intercept_,
        target_name='ΔE (RS–ZB)',
        clean_to_original_map=sisso.sym_clean_to_original_map_
    )
    print('\n===== Descriptor (D={}) ====='.format(D))
    print(formula_str)

## 4. Evaluate descriptor performance

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Helper: transform X once (cached)
phiX = sisso._transform_X(X)

for D, model_info in sisso.models_by_dim_.items():
    coef = model_info['coef'].flatten()
    if sisso.fix_intercept_:
        intercept = 0.0
        weights = coef
    else:
        intercept, *weights = coef
        weights = np.array(weights)
    X_D = phiX[model_info['features']]
    y_pred = intercept + X_D.values.dot(weights)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    print(f'D={D} -> RMSE={rmse:.4f} eV, R^2={r2:.3f}')

## 5. Conclusion
pySISSO successfully recovered interpretable low-dimensional descriptors that capture the stability trend between Rocksalt and Zincblende structures. Use the formulas and metrics above to compare with SISSO++ results.